In [188]:
import os
import pandas as pd
from dateutil.relativedelta import relativedelta
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import sklearn.model_selection
import graphviz

os.chdir(os.path.dirname(os.path.abspath(__file__)))

# データの読み込み
# 顧客行動データ
customer = pd.read_csv("input/join_customer_data.csv")
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_end_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [189]:
# 顧客毎の月別の利用回数データ
monthly_use_log = pd.read_csv("input/use_log_months.csv")
monthly_use_log.head()

,customer_id,usemonth,count
0,AS002855,2018-04,4
1,AS002855,2018-05,5
2,AS002855,2018-06,5
3,AS002855,2018-07,5
4,AS002855,2018-08,3


In [190]:
# 当月と過去1ヶ月の利用回数を集計したデータの作成
# 6ヶ月未満で退会する顧客もいるため、予測に用いる過去の利用履歴データの期間を第４章より短くする
year_months = list(monthly_use_log["usemonth"].unique())

use_log = pd.DataFrame()

# 過去1ヶ月のデータがある2018年5月以降のデータで作成する
for i in range(1, len(year_months)):
    tmp = monthly_use_log[monthly_use_log["usemonth"] == year_months[i]].copy()
    tmp.rename(columns={"count": "count_0"}, inplace=True)

    tmp_before = monthly_use_log[
        monthly_use_log["usemonth"] == year_months[i - 1]
    ].copy()
    tmp_before.drop("usemonth", axis=1, inplace=True)
    tmp_before.rename(columns={"count": "count_1"}, inplace=True)

    tmp = pd.merge(tmp, tmp_before, how="left", on="customer_id")

    use_log = pd.concat([use_log, tmp], ignore_index=True)

use_log.head()

,customer_id,usemonth,count_0,count_1
0,AS002855,2018-05,5,4.0
1,AS009373,2018-05,4,3.0
2,AS015233,2018-05,7,NaN
3,AS015315,2018-05,3,6.0
4,AS015739,2018-05,5,7.0


In [191]:
# 退会前月の退会顧客データを作成する
# 前提： 退会前月に退会を申し込み、翌月に退会する規則がある

# 1.退会済みの顧客のみのデータを作成する
exit_customer = customer[customer["is_deleted"] == 1].copy()

# 2.退会日（end_date）の1ヶ月前の年月データ（exit_month）列を作成する
# 2-1.退会日（end_date）の1ヶ月前の日付データ（exit_date）列を作成する
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
exit_customer.reset_index(drop=True, inplace=True)

for i in range(len(exit_customer)):
    exit_customer.loc[i, "exit_date"] = exit_customer.loc[
        i, "end_date"
    ] - relativedelta(months=1)

# 2-2.exit_dateをdatetime型に変換する
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])

# 2-3.exit_dateの日付を年月に変換したデータ列（exit_month）を作成する
exit_customer["exit_month"] = exit_customer["exit_date"].dt.strftime("%Y-%m")

# 3.顧客行動データ（use_log）と作成した退会済みの顧客の行動データ（exit_customer）を結合する
exit_use_log = pd.merge(
    use_log,
    exit_customer,
    how="left",
    left_on=["customer_id", "usemonth"],
    right_on=["customer_id", "exit_month"],
)

exit_use_log.drop("exit_month", axis=1, inplace=True)
print(len(use_log))

exit_use_log.head()

33851


,customer_id,usemonth,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_end_date,membership_period,exit_date
0,AS002855,2018-05,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,AS009373,2018-05,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,AS015233,2018-05,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,AS015315,2018-05,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,AS015739,2018-05,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [192]:
# 3.退会済みかつ退会前月の顧客の行動データのみを残す
# 3-1.nameデータに欠損値がある行を削除する
#   exit_customerの元になっているcustomerデータはend_date以外に欠損はないので、
#   end_date以外のデータ（ここではnameを指定した）が欠損している行は退会済みの顧客の退会前月以外のデータである
exit_use_log = exit_use_log.dropna(subset=["name"])

# 3-2.結合結果の確認
#  退会済みの1顧客につき退会前月データは1つのみなので、作成したデータでcustomer_idの重複がないことを確かめる
print(len(exit_use_log))
print(len(exit_use_log["customer_id"].unique()))
exit_use_log.head()

1104
1104


,customer_id,usemonth,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_end_date,membership_period,exit_date
19,AS055680,2018-05,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,AS169823,2018-05,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,AS305860,2018-05,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,AS363699,2018-05,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,AS417696,2018-05,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


In [193]:
# 継続顧客のデータを作成する
# 1.継続中の顧客のみの顧客行動データを作成
continue_customer = customer[customer["is_deleted"] == 0].copy()

# 2.全ての顧客（退会済みと継続中）の月別利用回数データと継続中の顧客行動データを結合
continue_use_log = pd.merge(use_log, continue_customer, on="customer_id", how="left")
print(len(continue_use_log))

# 3.退会済みの顧客行動データを削除
continue_use_log = continue_use_log.dropna(subset=["name"])
print(len(continue_use_log))

33851
27422


In [194]:
# 退会顧客と継続顧客のデータ数を揃えるため、継続顧客データをアンダーサンプリングする
# 1.データをシャッフルする
continue_use_log = continue_use_log.sample(frac=1, ignore_index=True)

# 2.customer_idが重複しているデータは最初のデータのみを残す
continue_use_log.drop_duplicates(subset="customer_id", inplace=True)

print(len(continue_use_log))
continue_use_log.head()

2842


,customer_id,usemonth,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_end_date,membership_period
0,GD699747,2019-03,5,5.0,XXXX,C01,M,2017-08-01,NaN,CA2,...,オールタイム,10500.0,入会費半額,5.333333,5.5,7.0,2.0,1.0,2019-04-30,20.0
1,TS020727,2019-03,5,8.0,XX,C01,M,2017-05-01,NaN,CA1,...,オールタイム,10500.0,通常,6.083333,6.0,8.0,3.0,1.0,2019-04-30,23.0
2,AS345508,2018-05,6,5.0,XXXX,C02,M,2017-07-01,NaN,CA2,...,デイタイム,7500.0,入会費半額,5.833333,6.0,8.0,3.0,1.0,2019-04-30,21.0
3,TS949361,2018-10,7,11.0,XXXX,C03,M,2018-06-13,NaN,CA2,...,ナイト,6000.0,入会費半額,7.900000,8.0,11.0,5.0,1.0,2019-04-30,10.0
4,IK972922,2018-08,7,3.0,XXXX,C02,F,2016-11-01,NaN,CA1,...,デイタイム,7500.0,通常,5.250000,5.5,8.0,2.0,1.0,2019-04-30,29.0


In [195]:
# 継続顧客データと退会顧客データを結合する
predict_data = pd.concat([continue_use_log, exit_use_log], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,customer_id,usemonth,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_end_date,membership_period,exit_date
0,GD699747,2019-03,5,5.0,XXXX,C01,M,2017-08-01,NaT,CA2,...,10500.0,入会費半額,5.333333,5.5,7.0,2.0,1.0,2019-04-30,20.0,NaT
1,TS020727,2019-03,5,8.0,XX,C01,M,2017-05-01,NaT,CA1,...,10500.0,通常,6.083333,6.0,8.0,3.0,1.0,2019-04-30,23.0,NaT
2,AS345508,2018-05,6,5.0,XXXX,C02,M,2017-07-01,NaT,CA2,...,7500.0,入会費半額,5.833333,6.0,8.0,3.0,1.0,2019-04-30,21.0,NaT
3,TS949361,2018-10,7,11.0,XXXX,C03,M,2018-06-13,NaT,CA2,...,6000.0,入会費半額,7.900000,8.0,11.0,5.0,1.0,2019-04-30,10.0,NaT
4,IK972922,2018-08,7,3.0,XXXX,C02,F,2016-11-01,NaT,CA1,...,7500.0,通常,5.250000,5.5,8.0,2.0,1.0,2019-04-30,29.0,NaT


In [196]:
# 予測月の初日時点での会員期間を計算する
# 1.予測月の初日データ(now_date)列を作成する
predict_data["now_date"] = pd.to_datetime(predict_data["usemonth"], format="%Y-%m")

# 2.start_date列をdatetime型に変換する
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

# 3.予測月までの会員期間（membership_period: now_dateとstart_dateの差分）を計算する)
predict_data["membership_period"] = 0

for i in range(len(predict_data)):
    delta = relativedelta(
        predict_data.loc[i, "now_date"], predict_data.loc[i, "start_date"]
    )
    predict_data.loc[i, "membership_period"] = int(delta.years * 12 + delta.months)

predict_data.head()

,customer_id,usemonth,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flg,calc_end_date,membership_period,exit_date,now_date
0,GD699747,2019-03,5,5.0,XXXX,C01,M,2017-08-01,NaT,CA2,...,入会費半額,5.333333,5.5,7.0,2.0,1.0,2019-04-30,19,NaT,2019-03-01
1,TS020727,2019-03,5,8.0,XX,C01,M,2017-05-01,NaT,CA1,...,通常,6.083333,6.0,8.0,3.0,1.0,2019-04-30,22,NaT,2019-03-01
2,AS345508,2018-05,6,5.0,XXXX,C02,M,2017-07-01,NaT,CA2,...,入会費半額,5.833333,6.0,8.0,3.0,1.0,2019-04-30,10,NaT,2018-05-01
3,TS949361,2018-10,7,11.0,XXXX,C03,M,2018-06-13,NaT,CA2,...,入会費半額,7.900000,8.0,11.0,5.0,1.0,2019-04-30,3,NaT,2018-10-01
4,IK972922,2018-08,7,3.0,XXXX,C02,F,2016-11-01,NaT,CA1,...,通常,5.250000,5.5,8.0,2.0,1.0,2019-04-30,21,NaT,2018-08-01


In [197]:
# 欠損値を除去する
# 1.欠損値の数を確認する
predict_data.isnull().sum()

customer_id             0
usemonth                0
count_0                 0
count_1               256
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_end_date           0
membership_period       0
exit_date            2842
now_date                0
dtype: int64

In [198]:
# 2.count_1のデータが欠損している行を除外する
predict_data.dropna(subset=["count_1"], inplace=True)

# 3.除外できたことを確認する
predict_data.isnull().sum()

customer_id             0
usemonth                0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2638
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_end_date           0
membership_period       0
exit_date            2638
now_date                0
dtype: int64

In [199]:
# 文字列型の変数を処理できるように整形する
# 1.予測に用いるデータ列のみ残す
target_columns = [
    "campaign_name",
    "class_name",
    "gender",
    "count_1",
    "routine_flg",
    "membership_period",
    "is_deleted",
]
predict_data = predict_data[target_columns]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,membership_period,is_deleted
0,入会費半額,オールタイム,M,5.0,1.0,19,0.0
1,通常,オールタイム,M,8.0,1.0,22,0.0
2,入会費半額,デイタイム,M,5.0,1.0,10,0.0
3,入会費半額,ナイト,M,11.0,1.0,3,0.0
4,通常,デイタイム,F,3.0,1.0,21,0.0


In [200]:
# 2.カテゴリー変数をダミー変数化する
predict_data = pd.get_dummies(predict_data, dtype=int)
predict_data.head()

,count_1,routine_flg,membership_period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,19,0.0,1,0,0,1,0,0,0,1
1,8.0,1.0,22,0.0,0,0,1,1,0,0,0,1
2,5.0,1.0,10,0.0,1,0,0,0,1,0,0,1
3,11.0,1.0,3,0.0,1,0,0,0,0,1,0,1
4,3.0,1.0,21,0.0,0,0,1,0,1,0,1,0


In [201]:
# 3.不要なダミー変数を削除する
predict_data.drop(
    columns=["campaign_name_通常", "class_name_ナイト", "gender_M"], inplace=True
)
predict_data.head()

,count_1,routine_flg,membership_period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,19,0.0,1,0,1,0,0
1,8.0,1.0,22,0.0,0,0,1,0,0
2,5.0,1.0,10,0.0,1,0,0,1,0
3,11.0,1.0,3,0.0,1,0,0,0,0
4,3.0,1.0,21,0.0,0,0,0,1,1


In [202]:
# 決定木を用いて退会予測モデルを作成する
# 1.退会顧客と継続顧客が同数含まれるデータを作成する
exit_predict_data = predict_data[predict_data["is_deleted"] == 1].copy()
continue_predict_data = (
    predict_data[predict_data["is_deleted"] == 0]
    .copy()
    .sample(n=len(exit_predict_data))
)

X = pd.concat([exit_predict_data, continue_predict_data], ignore_index=True)
y = X["is_deleted"].copy()
X.drop(columns=["is_deleted"], inplace=True)

# 2.学習用データと評価用データに分割する
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

# 3.モデルを作成する
model = DecisionTreeClassifier(random_state=0)
# 3-1.訓練データからモデルを構築する
model.fit(X_train, y_train)
# 3-2.与えられた入力に対する予測結果を返す
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.
 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1.

In [203]:
# 4.予測結果と正解結果の比較を行う
results_test = pd.DataFrame({"y_test": y_test, "y_pred": y_test_pred})
results_test.head()

,y_test,y_pred
318,1.0,1.0
1406,0.0,0.0
1453,0.0,0.0
1361,0.0,0.0
1807,0.0,0.0


In [204]:
# 予測モデルの評価
correct = len(results_test[results_test["y_test"] == results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9182509505703422


In [205]:
print(f"test score: {model.score(X_test, y_test)}")
print(f"train score: {model.score(X_train, y_train)}")

test score: 0.9182509505703422
train score: 0.9797211660329531


In [206]:
# モデルパラメータの調整
# max_depthを5にする
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)

print(f"test score: {model.score(X_test, y_test)}")
print(f"train score: {model.score(X_train, y_train)}")

test score: 0.9182509505703422
train score: 0.9239543726235742


In [207]:
# モデルに寄与している変数の確認
importance = pd.DataFrame(
    {"feature_names": X.columns, "coeeficient": model.feature_importances_}
)
importance

,feature_names,coeeficient
0,count_1,0.357484
1,routine_flg,0.122869
2,membership_period,0.519457
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000190
6,class_name_デイタイム,0.000000
7,gender_F,0.000000


In [208]:
# 決定木の可視化
# 1.DOTフォーマットで決定木を出力する
dot = export_graphviz(model, out_file=None, feature_names=X.columns)
# 2.DOTソースコードをgraphvizで画像生成できるよう処理する
graph = graphviz.Source(dot)

if not os.path.exists("output"):
    # ディレクトリが存在しない場合、ディレクトリを作成する
    os.makedirs("output")
# 3.ソースコードをファイルに保存しgraphvizで画像生成する
graph.view("output/image", cleanup=True)

'output/image.pdf'

In [209]:
# 顧客の退会を予測する
count_1 = 5
routing_flg = 1
membership_period = 10
campaign_name = "通常"
class_name = "ナイト"
gender = "M"

if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]

if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]

if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]

input_data = [count_1, routing_flg, membership_period]
input_data += campaign_name_list + class_name_list + gender_list
input_data

[5, 1, 10, 0, 0, 0, 0, 0]

In [210]:
X_pred = pd.DataFrame([input_data], columns=X.columns)
print(model.predict(X_pred))
print(model.predict_proba(X_pred))

[0.]
[[0.8 0.2]]
